In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from scipy.stats import entropy

%cd '/content/drive/MyDrive/Colab Notebooks/media-agenda'
import util
from util import DocType, Source, OptimalKClustersConfig

# metadata_file = '/content/drive/MyDrive/Colab Notebooks/media-agenda/data/sentence_cluster_sentiment_dict.json'
metadata_file = '/content/drive/MyDrive/Colab Notebooks/media-agenda/data/sentence_cluster_sentiment_dict_with_cosine_similarity_for_15_clusters.csv'
sid_file = '/content/drive/MyDrive/Colab Notebooks/media-agenda/data/desired_sentence_ids.npy'
rsfile = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/revised_all_sentences_str.json'
rifile = '/content/drive/My Drive/Colab Notebooks/media-agenda/data/revised_all_sentences_ind.json'

%cd '/content/drive/MyDrive/Colab Notebooks/media-agenda/data'

/content/drive/.shortcut-targets-by-id/1jko4SIvsKFTxv155O0DV1wHZ_jS75VfX/Colab Notebooks/media-agenda
/content/drive/.shortcut-targets-by-id/1jko4SIvsKFTxv155O0DV1wHZ_jS75VfX/Colab Notebooks/media-agenda/data


In [ ]:
# load all sentences
all_sentences = np.array(util.load_sentences(rsfile))

# load sentence id for sentences with character length < 15
desired_sid = np.load(sid_file)
print('Loaded desired id: ', len(desired_sid))

# load sentences assignement as dataframe
# df = pd.read_json(metadata_file)
df = pd.read_csv(metadata_file)
print('Loaded sentence assigments: ', df.shape[0])
display(df.head())

Sentences file - loaded
Done - appended all sentences
Number of tokenized sentences from corpus: 353786
Loaded desired id:  328091
Loaded sentence assigments:  328091


,Unnamed: 0,sentence_id,source,article_id,comment_id,posting_time,cluster,sentiw_sentiment,textblob_polarity,textblob_subjectivity,cosine_similarity
0,0,0,nytimes,0,-1,2005-11-01 00:00:00,1,0.12500,-0.017045,0.518182,0.318689
1,1,1,nytimes,0,-1,2005-11-01 00:00:00,1,-0.12500,0.337500,0.675000,0.396096
2,2,2,nytimes,0,-1,2005-11-01 00:00:00,3,0.07500,0.085714,0.192857,0.537876
3,3,3,nytimes,0,-1,2005-11-01 00:00:00,1,0.50000,0.025000,0.250000,0.433355
4,4,4,nytimes,0,-1,2005-11-01 00:00:00,1,-0.03125,-0.050000,0.291667,0.384546


In [ ]:
def get_topic_distribution_by_article_save_to_csv(source, article_id):
  sub_df = df[(df.source == source) & (df.textblob_polarity != 0)]
  article_df = df[(df.source == source) & (df.textblob_polarity != 0) & (df.comment_id == DocType.NON_COMMENT.value)]
  comment_df = df[(df.source == source) & (df.textblob_polarity != 0) & (df.comment_id != DocType.NON_COMMENT.value)]

  num_of_article = len(article_df.article_id.unique())
  num_of_article_with_comment = len(comment_df.article_id.unique())
  print('For {}: {} articles, {} with comments'.format(source, num_of_article, num_of_article_with_comment))
  
  optimal_clusters_df = pd.DataFrame({'cluster': range(OptimalKClustersConfig.k_with_garbage),
                                      'topic': OptimalKClustersConfig.clusters_with_garbage,
                                      'topic2': OptimalKClustersConfig.clusters_with_garbage_nonum})
  print('Optimal clusters for k = 15')
  display(optimal_clusters_df)

  if article_id != None: 

    result_df = pd.DataFrame({'article_sentence_count': article_df[article_df.article_id == article_id].groupby('cluster').sentence_id.count(),
                              # 'article_sentiment_median': article_df[article_df.article_id == article_id].groupby('cluster').textblob_polarity.median(),
                              # 'article_sentiment_iqr': article_df[article_df.article_id == article_id].groupby('cluster').textblob_polarity.quantile(0.75) - article_df[article_df.article_id == article_id].groupby('cluster').textblob_polarity.quantile(0.25),
                              'comment_sentence_count': comment_df[comment_df.article_id == article_id].groupby('cluster').sentence_id.count(),
                              # 'comment_sentiment_median': comment_df[comment_df.article_id == article_id].groupby('cluster').textblob_polarity.median(),
                              # 'comment_sentiment_iqr': comment_df[comment_df.article_id == article_id].groupby('cluster').textblob_polarity.quantile(0.75) - comment_df[comment_df.article_id == article_id].groupby('cluster').textblob_polarity.quantile(0.25),
                              'article_sentence_id_list': article_df[article_df.article_id == article_id].groupby('cluster').sentence_id.apply(list),
                              'comment_sentence_id_list': comment_df[comment_df.article_id == article_id].groupby('cluster').sentence_id.apply(list)
                              }).reset_index()
                  
    result_df = optimal_clusters_df.merge(result_df, how = 'left', validate = 'one_to_one')

    # display(result_df)

    article_result_df = result_df[(pd.notna(result_df.article_sentence_count))]
    comment_result_df = result_df[(pd.notna(result_df.comment_sentence_count))]

    clustered_sentences = []
    # for i in OptimalKClustersConfig.valid_cluster_index:
    for i in range(OptimalKClustersConfig.k_with_garbage):
      if not article_result_df[article_result_df.cluster == i].empty:
        article_sentence_indeces = np.concatenate(article_result_df[article_result_df.cluster == i].article_sentence_id_list.values).ravel()
        for j in article_sentence_indeces:
          clustered_sentences.append((source, 
                                      article_id,
                                      'Article', 
                                      i,
                                      OptimalKClustersConfig.clusters_with_garbage[i], 
                                      # cluster_label_for_15[i],
                                      all_sentences[j],
                                      df[df.sentence_id == j]['sentence_id'].values[0],
                                      df[df.sentence_id == j].cosine_similarity.values[0],
                                      df[df.sentence_id == j].textblob_polarity.values[0]))
          
      if not comment_result_df[comment_result_df.cluster == i].empty:
        comment_sentence_indeces = np.concatenate(comment_result_df[comment_result_df.cluster == i].comment_sentence_id_list.values).ravel()
        for k in comment_sentence_indeces:
          clustered_sentences.append((source, 
                                    article_id,
                                    'Comment', 
                                    i,
                                    OptimalKClustersConfig.clusters_with_garbage[i],
                                    # cluster_label_for_15[i],
                                    all_sentences[k],
                                    df[df.sentence_id == k]['sentence_id'].values[0],
                                    df[df.sentence_id == k]['cosine_similarity'].values[0],
                                    df[df.sentence_id == k].textblob_polarity.values[0]))
    
    clustered_sentences_df = pd.DataFrame(clustered_sentences, columns = ['source', 'article_id', 'doc_type', 'cluster', 'topic', 'sentence', 'sentence_id', 'cosine_similarity', 'textblob_polarity'])
    pd.set_option('display.max_rows', None, 'display.max_colwidth', None)
    # displot(clustered_sentences_df)
    # display(clustered_sentences_df.sort_values(by = 'cosine_similarity', ascending = False).sort_values(by = ['cluster', 'doc_type']))
        
    return_df = clustered_sentences_df.sort_values(by = 'cosine_similarity', ascending = False).sort_values(by = ['cluster', 'doc_type'])
    
    return return_df
    # boxplot(sub_df[sub_df.article_id == article_id], article_id, source, magnified = True)

In [22]:
all_cluster_and_type = pd.concat([pd.DataFrame({'cluster': range(OptimalKClustersConfig.k_with_garbage), 
                                                'topic': OptimalKClustersConfig.clusters_with_garbage,
                                                'doc_type': 'Article'}),
                                  pd.DataFrame({'cluster': range(OptimalKClustersConfig.k_with_garbage), 
                                                'topic': OptimalKClustersConfig.clusters_with_garbage,
                                                'doc_type': 'Comment'})])

In [23]:
article_id = 175
nytimes_175 = get_topic_distribution_by_article_save_to_csv(source = Source.NYTIMES, article_id = article_id)
nytimes_175 = pd.merge(all_cluster_and_type, nytimes_175, how = 'outer', on = ['cluster', 'topic', 'doc_type'])
nytimes_175.to_csv('nytimes_175.csv')

For nytimes: 327 articles, 99 with comments
Optimal clusters for k = 15


,cluster,topic,topic2
0,0,Environment (pesticides & fertilizers),Environment (pesticides & fertilizers)
1,1,Retailers,Retailers
2,2,Garbage 1,Garbage
3,3,GMO & organic,GMO & organic
4,4,Garbage 2,Garbage
5,5,Food products & taste,Food products & taste
6,6,Food safety,Food safety
7,7,Research,Research
8,8,Health & nutrition,Health & nutrition
9,9,Garbage 3,Garbage


In [24]:
article_id = 92
spiegel_92 = get_topic_distribution_by_article_save_to_csv(source = Source.SPIEGEL, article_id = article_id)
spiegel_92 = pd.merge(all_cluster_and_type, spiegel_92, how = 'outer', on = ['cluster', 'topic', 'doc_type'])
spiegel_92.to_csv('spiegel_92.csv')

For spiegel: 150 articles, 61 with comments
Optimal clusters for k = 15


,cluster,topic,topic2
0,0,Environment (pesticides & fertilizers),Environment (pesticides & fertilizers)
1,1,Retailers,Retailers
2,2,Garbage 1,Garbage
3,3,GMO & organic,GMO & organic
4,4,Garbage 2,Garbage
5,5,Food products & taste,Food products & taste
6,6,Food safety,Food safety
7,7,Research,Research
8,8,Health & nutrition,Health & nutrition
9,9,Garbage 3,Garbage
